In [38]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np
import json

In [246]:
root = '/Volumes/SD/Data/semantic_annotations/'

1. Find Json file of a folder

In [247]:
json_files = [ x for x in os.listdir(root) if x.endswith("json") ]

In [248]:
len(json_files)

66261

2. Load and parse a single Json file

In [249]:
with open(root + json_files[1]) as json_file:
    data = json.load(json_file)

In [250]:
ele_type = []
ele_coor = []
for c in data['children']:
    ele_type.append(c['componentLabel'])
    ele_coor.append(c['bounds'])

3. Process all J

In [251]:
def read_ele(file_names, root, max_element_num = 1000):
    all_ele_type = []
    all_ele_coor = []
    for f in file_names:
        with open(root + f) as json_file:
            data = json.load(json_file)
            if not max_element_num < len(data['children']):
                ele_type = []
                ele_coor = []
                for c in data['children']:
                    ele_type.append(c['componentLabel'])
                    ele_coor.append(c['bounds'])
                all_ele_type.append(ele_type)
                all_ele_coor.append(ele_coor)
    return (all_ele_type, all_ele_coor)

In [252]:
t, c = read_ele(json_files, root, max_element_num = 9)

4. One-hot encoder

In [253]:
# Get Unique Type, number of unique types, element to int
uni_types = list(set([x for ele in t for x in ele]))
max_len = len(uni_types)
ele_dict = {v:i for i, v in enumerate(uni_types)}

In [254]:
# One-hot encoding
def onehot_encoder(ele, ele_dict, max_len):
    l = [0] * max_len
    idx = ele_dict[ele]
    l[idx] = 1
    return l
onehot_encoder('Image', ele_dict, max_len)

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

5. Process All the data

In [261]:
t_ = [x for x in t if x != []]
c_ = [x for x in c if x != []]

In [262]:
ele_all = []
count = 0
for ele, coor in zip(t_,c_):
    ele_enc = [onehot_encoder(e, ele_dict, max_len) for e in ele]
    ele_merge = [np.array(e+c) for e,c in zip(ele_enc, coor)]
    fill_len = 9 - len(ele)
    try:
        if fill_len > 0:
            ele_fill = np.zeros((fill_len,max_len+4))
            ele_merge = np.concatenate((ele_merge, ele_fill))
    except:
        print(count)
    count += 1
    ele_all.append(ele_merge)

6. DataLoader

In [263]:
x = torch.from_numpy(np.array(ele_all))
y = torch.ones(x.size(0))

In [264]:
dataset = TensorDataset(x, y)
data_loader = DataLoader(dataset, batch_size=100, shuffle=True)